In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import glob
import gc
import functools
from FaronF1 import F1Optimizer
from multiprocessing import cpu_count,Pool
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.ensemble import RandomForestRegressor
pd.options.display.max_columns=400
pd.options.display.max_rows=400


In [2]:
ROOT_DIR = '../data/instacart/files/'
out_of_fold = pd.read_csv(ROOT_DIR+'out_of_fold_preds.csv', usecols=['user_id', 'order_id', 'product_id', 'predicted'])
test_preds = pd.read_csv(ROOT_DIR+'test_preds.csv')
test_preds.rename(columns={'preds':'predicted'}, inplace=True)
test_preds.sort_values(['user_id', 'predicted'], ascending=[True, False], inplace=True)

In [3]:
def f1_maximization(order_id):
    p_vals = test_preds.loc[test_preds.order_id==order_id, 'predicted'].values
    k,is_none,expected_F1 = F1Optimizer.maximize_expectation(p_vals)
    return k,is_none,expected_F1

In [4]:
p = Pool(cpu_count())
expected_F1 = p.map(f1_maximization, list(test_preds.order_id.unique()))

In [8]:
write_dict = {}
for i,order in enumerate(test_preds.order_id.unique()):
    bask_size = expected_F1[i][0]
    if bask_size==0:
        write_dict[order] = 'None'
    else:
        products = test_preds.loc[test_preds.order_id==order, 'product_id'].values[:bask_size].astype(np.str)
        write_dict[order] = ' '.join(products)
    

In [19]:
write_dict = pd.DataFrame.from_dict(write_dict, orient='index')


In [20]:
write_dict.reset_index(inplace=True)
write_dict.columns = ['order_id', 'products']
write_dict.to_csv(ROOT_DIR+'maximize_expectation.csv', index=False)